In [1]:
import pandas as pd
import ast

In [2]:
operation_type = "rent"
date_or_unioned = "unioned"

In [3]:
file_name = f"{date_or_unioned}-lisbon-listings-for-{operation_type}.csv"

In [4]:
df = pd.read_csv(f'../data/idealista/raw/{file_name}', index_col='propertyCode')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [10]:
df = df.drop(columns=['thumbnail', 'numPhotos', 'operation', 'hasVideo', 'hasPlan', 'has3DTour', 'has360', 'hasStaging', 'externalReference', 'detailedType', 'suggestedTexts'])

In [ ]:
df.columns

In [12]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.floor.unique()

In [ ]:
df[df.floor == 'en']

In [17]:
df.floor = df.floor.str.replace('bj', '0')
df.floor = df.floor.str.replace('-', '')
df.floor = df.floor.str.replace('st', '-1')
df.floor = df.floor.str.replace('ss', '-1')
df.floor = df.floor.str.replace('en', '-1')
df.floor = df.floor.astype(float)

In [ ]:
df.floor.describe()

In [ ]:
df.priceInfo.unique()

In [20]:
# Function to parse the price and coalesce the formerPrice
def extract_former_price(price_str):
    # Parse the string to a dictionary
    price_dict = ast.literal_eval(price_str)
    price_info = price_dict.get('price', {})
    amount = price_info.get('amount')  # Current price
    price_drop_info = price_info.get('priceDropInfo', {})
    former_price = price_drop_info.get('formerPrice')  # Former price, if available
    # Coalesce formerPrice with the current price
    return former_price if former_price is not None else amount

# Apply the function to create the formerPrice column
df['formerPrice'] = df['priceInfo'].apply(extract_former_price)
df['priceChange'] = df['formerPrice'] - df['price']
df = df.drop(columns=['priceInfo'])

In [ ]:
df['formerPrice']

In [ ]:
df.head()

In [ ]:
df.parkingSpace.unique()

In [24]:
df['parkingSpace'] = df['parkingSpace'].fillna('{}')
df['parsed'] = df['parkingSpace'].apply(ast.literal_eval)
df['hasParkingSpace'] = df['parsed'].apply(lambda x: x.get('hasParkingSpace', False))
df['isParkingSpaceIncludedInPrice'] = df['parsed'].apply(lambda x: x.get('isParkingSpaceIncludedInPrice', False))
df.drop(columns=['parkingSpace', 'parsed'], inplace=True)

In [ ]:
df.head()

In [ ]:
df[df.hasLift.isnull()].head()

In [ ]:
df.columns

In [28]:
df = df[df['price'] > 0]

In [29]:
import pandas as pd

def remove_outliers(df, variables, thresholds=None):
    """
    Removes outliers from specified variables using IQR or domain-based thresholds.
    
    Parameters:
    df (pd.DataFrame): The dataset.
    variables (list): List of variables to check for outliers.
    thresholds (dict): Optional custom lower and upper bounds for specific variables.
                       Format: {'variable': {'lower': value, 'upper': value}}
    
    Returns:
    pd.DataFrame: Dataset with outliers removed.
    """
    df_cleaned = df.copy()
    
    for variable in variables:
        # Check for custom thresholds
        if thresholds and variable in thresholds:
            lower = thresholds[variable].get('lower', None)
            upper = thresholds[variable].get('upper', None)
        else:
            # Default to IQR if no custom threshold provided
            Q1 = df_cleaned[variable].quantile(0.25)
            Q3 = df_cleaned[variable].quantile(0.75)
            IQR = Q3 - Q1
            lower = Q1 - 1.5 * IQR
            upper = Q3 + 1.5 * IQR
        
        # Remove outliers
        df_cleaned = df_cleaned[(df_cleaned[variable] >= lower) & (df_cleaned[variable] <= upper)]
    
    return df_cleaned

In [ ]:
df.select_dtypes(include='number').columns

In [31]:
variables_with_outliers = ['price', 'size', 'rooms', 'bathrooms']

In [32]:
# Remove outliers for all specified variables
df_cleaned = remove_outliers(df, variables_with_outliers)

In [ ]:
df_cleaned.info()

In [34]:
df_cleaned.to_csv(f'../data/idealista/cleaned/{file_name}', index=True)